In [ ]:
from osgeo import gdal, osr, ogr, gdalconst
import os
import numpy as np
from shapely.geometry import mapping, shape
from shapely.wkt import loads
from shapely.geometry import Polygon
import json
import xml.etree.ElementTree as ET 
import glob
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
# from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date


In [ ]:
def plot_sentinel2(path, ref_tile):
    list1 = glob.glob(path+'/*{}*'.format(ref_tile))
    list2 = [glob.glob(x+"/GRANULE/*/IMG_DATA/R60m/*_TCI_60m.jp2") for x in list1]

    ncols=4
    fig = plt.figure(figsize=(10,15))
    gs = gridspec.GridSpec(ncols=ncols, nrows=len(list2)//ncols +1) #width_ratios= [1, 1], #height_ratios=[1, 1],

    medians = []
    masks = []

    for count, id_ in enumerate(list2):
        if len(id_) > 0:
            ds = gdal.Open(id_[0])
            array = ds.ReadAsArray()
            array = array.transpose().swapaxes(0, 1)

            ax = plt.subplot(gs[count])

            ax.imshow(array)
            ax.axis('off')

    #         plt.imshow(array)
            print(count, id_[0].split('/')[-6])
            ax.set_title(count)

            mask_ = (array == 0).all(axis=-1)

            medians.append(array)
            try:

                ds = gdal.Open(id_[0].replace('TCI_60m','SCL_60m'))
                arrayscl = ds.ReadAsArray()
                arrayscl = arrayscl.transpose().swapaxes(0, 1)
                mask_ = np.logical_or(mask_, arrayscl==3)
                mask_ = np.logical_or(mask_, arrayscl == 11)
                mask_ = np.logical_or(mask_, arrayscl == 6)


                id_cld = glob.glob(id_[0].split('/IMG_DATA/')[0]+'/QI_DATA/*CLD_60m.jp2')[0]
                array_cld = ds.ReadAsArray()
                array_cld = array_cld.transpose().swapaxes(0, 1)
                mask_ = np.logical_or(mask_, array_cld > 5)
            except IndexError:
                print('error in cld, or SCL',count)

            mask_ = np.repeat(mask_[...,np.newaxis], 3, axis=-1)
            masks.append(mask_)



    mask_ = np.stack(masks)
    median_ = np.ma.masked_array(np.stack(medians), mask_)
    median_ = np.ma.median(median_,axis=0) /255.0
    ax = plt.subplot(gs[-1])
    ax.imshow(median_)

    ax.axis('off')
    ax.set_title('median')

    plt.show()

    plt.imshow(median_)
    plt.axis('off')
    plt.title('median')

In [ ]:
def get_median_sentinel2(path, ref_tile):
    list1 = glob.glob(path+'/*{}*'.format(ref_tile))
    list2 = [glob.glob(x+"/GRANULE/*/IMG_DATA/R60m/*_TCI_60m.jp2") for x in list1]

    medians = []
    masks = []

    for count, id_ in enumerate(list2):
        if len(id_) > 0:
            ds = gdal.Open(id_[0])
            array = ds.ReadAsArray()
            array = array.transpose().swapaxes(0, 1)


#            print(count, id_[0].split('/')[-6])

            mask_ = (array == 0).all(axis=-1)

            medians.append(array)
            try:

                ds = gdal.Open(id_[0].replace('TCI_60m','SCL_60m'))
                
                arrayscl = ds.ReadAsArray()
                arrayscl = arrayscl.transpose().swapaxes(0, 1)
                mask_ = np.logical_or(mask_, arrayscl==3)
                mask_ = np.logical_or(mask_, arrayscl == 11)
                mask_ = np.logical_or(mask_, arrayscl == 6)


                id_cld = glob.glob(id_[0].split('/IMG_DATA/')[0]+'/QI_DATA/*CLD_60m.jp2')[0]
                ds = gdal.Open(id_cld)
                array_cld = ds.ReadAsArray()
                array_cld = array_cld.transpose().swapaxes(0, 1)
                mask_ = np.logical_or(mask_, array_cld > 5)
            except IndexError:
                print('error in cld, or SCL',count)

            mask_ = np.repeat(mask_[...,np.newaxis], 3, axis=-1)
            masks.append(mask_)



    mask_ = np.stack(masks)
    median_ = np.ma.masked_array(np.stack(medians), mask_)
    median_ = np.ma.median(median_,axis=0) /255.0
    
    return median_
    


In [ ]:
plot_sentinel2(path='/scratch/andresro/leon_work/barry_palm/data/2A/phillipines_2017/',
ref_tile='T51PXP')

In [ ]:
plot_sentinel2(path='/scratch/andresro/leon_work/barry_palm/data/2A/palmcountries_2017',
ref_tile='T50MNS')

In [ ]:
path = '/scratch/andresro/leon_igp/barry_palm/data/2A/palmcountries_2017'

# ref_tile='R018_T47NQA_'
ref_tile ='T50_MMD'

list1 = glob.glob(path+'/*{}*'.format(ref_tile))
list2 = [glob.glob(x+"/GRANULE/*/IMG_DATA/R60m/*_TCI_60m.jp2") for x in list1]


fig = plt.figure(figsize=(20,12))
gs = gridspec.GridSpec(ncols=4, nrows=len(list2)//4 +1) #width_ratios= [1, 1], #height_ratios=[1, 1],

    
    
for count, id_ in enumerate(list2):
    if len(id_) > 0:
        ds = gdal.Open(id_[0])
        array = ds.ReadAsArray()
        array = array.transpose().swapaxes(0, 1)
        
        ax = plt.subplot(gs[count])
    
        ax.imshow(array)
    
#         plt.imshow(array)
        print(count, id_[0].split('/')[-6])
        ax.set_title(count)
plt.show()

In [ ]:
path = '/scratch/andresro/leon_igp/barry_palm/data/2A/phillipines_2017'
# path = '/scratch/andresro/leon_igp/barry_palm/data/1C/phillipines_2017/PRODUCT'
ref_tile='R046_T50PNQ'


list1 = glob.glob(path+'/*{}*'.format(ref_tile))
list2 = [glob.glob(x+"/GRANULE/*/IMG_DATA/R60m/*_TCI_60m.jp2") for x in list1]


fig = plt.figure(figsize=(20,12))
gs = gridspec.GridSpec(ncols=4, nrows=len(list2)//4 +1) #width_ratios= [1, 1], #height_ratios=[1, 1],

    
    
for count, id_ in enumerate(list2):
    if len(id_) > 0:
        ds = gdal.Open(id_[0])
        array = ds.ReadAsArray()
        array = array.transpose().swapaxes(0, 1)
        
        ax = plt.subplot(gs[count])
    
        ax.imshow(array)
    
#         plt.imshow(array)
        print(count, id_[0].split('/')[-6])
        ax.set_title(count)
plt.show()

In [ ]:
path = '/scratch/andresro/leon_igp/barry_palm/data/2A/phillipines_2017'
# path = '/scratch/andresro/leon_igp/barry_palm/data/1C/phillipines_2017/PRODUCT'
ref_tile='R046_T50PNQ'


list1 = glob.glob(path+'/*{}*'.format(ref_tile))
# print(list1)
# list2 = [glob.glob(x+"/**/*.jp2", recursive=True) for x in list1]
list2 = [glob.glob(x+"/GRANULE/*/IMG_DATA/*/*.jp2", recursive=True) for x in list1]
for i in list2[0]:
    print(i.split('/')[-1])



In [ ]:
path = '/scratch/andresro/leon_igp/barry_palm/data/2A/palmcountries_2017'
ref_tile='_R118_T48NUG_'


list1 = glob.glob(path+'/*{}*'.format(ref_tile))

# print(list1)
list2 = [glob.glob(x+"/GRANULE/*/IMG_DATA/*/*.jp2", recursive=True) for x in list1]

for i in list2[0]:
    print(i.split('/')[-1])


In [ ]:

len(list2[0])